# Train using Nueral Network

In [1]:
#####################################################
#Library
print(" i am in library start")
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import psycopg2,os,csv

from IPython.display import display
pd.options.display.max_columns = None

from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout,Embedding
from keras.utils import np_utils
from keras.utils import plot_model

from sklearn.metrics import classification_report
 
from pickle import dump
import pickle
print(" i am in library end")

len_input_NN = 30  #this determined the number of Input to the nueral network model
#######################################################
def train_model(type_):

    #Connecting to Database
    print(" i am in connection database start")
    conn = psycopg2.connect("dbname=postgres user=postgres password=Msjobs@123")

    def select_data(type_):
        cur = conn.cursor()
        sql_command = "select value_,key_ from public.lct_ml_key_value where type_ ='" +type_+"';"
    #     sql_command = "select value_,key_ from public.lct_ml_key_value where type_ = 'location';"
        data = pd.read_sql(sql_command, conn)
        conn.commit()
        cur.close()
        return data  

    #Renaming the columns
    # type_ = 'location'
    test_df = select_data(type_).rename(columns={"value_": "data","key_": "result"})
    print(" i am in connection database end")
    #####################################################
    # Handling varity of characters in the upcoming inputs
    print(" i am in char_num building start")
#     listchars_ = ' "abcdefghijklmnopqrstvuwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890-=+_`~[]{}\|:;,.<>?/!@#$%^&*()ïáúãüóíéñÁäöôàÉ'
    listchars_ = ' "abcdefghijklmnopqrstvuwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890-=+_`~[]{}\|:;,.<>?/!@#$%^&*()'
    chars = sorted(list(set(listchars_)))

    char_to_integer = []
    for integer, char in enumerate(chars,start=1):
        char_to_integer.append((char, integer))
    char_to_integer = dict(char_to_integer)

    integer_to_char = []
    for integer, char in enumerate(chars,start=1):
        integer_to_char.append((integer,char))
    integer_to_char = dict(integer_to_char)
    print(" i am in char_num building end")
    #####################################################
    # Assigning Numeric to all the values
    print(" i am in assigning start")
#     len_input_NN = 30  #this determined the number of Input to the nueral network model
    test_df["data_int"]= ""

    for i in range(0, (len(test_df.data))):
        data_chr_int =[0]*len_input_NN
        data_ = str(test_df.data[i])
        i_ = 0 
        for char in data_:
            try:
                data_chr_int[i_] = char_to_integer[char]
            except:
                data_chr_int[i_] = 0
            if i_ >= len_input_NN-1:
                break
            i_+=1
        test_df["data_int"][i] = data_chr_int
    print(" i am in assigning end")

    #####################################################
    print(" i am in category start")
    # Assigning the category to index and index to category for predicting the category in future using index
    result_char = (list(set(test_df.result)))
    cat_to_integer = []
    integer_to_cat = []

    for integer, char in enumerate(result_char,start=0):
        cat_to_integer.append((char, integer))
    cat_to_integer = dict(cat_to_integer)

    for integer, char in enumerate(result_char,start=0):
        integer_to_cat.append((integer, char))
    integer_to_cat = dict(integer_to_cat)

    test_df["result_int"] = ""
    for i in range(0, (len(test_df.result))):
        test_df["result_int"][i] = cat_to_integer[test_df['result'][i]]
    print(" i am in category end")

    #####################################################
    print(" i am in preparing start")
    # Preparing the data for the Nueral Network model 
    X_train =[]

    for i in test_df.data_int:
        X_train.append(i)

    # Reshape X_train and normalize
    X_train = np.reshape(X_train, (len(X_train), len_input_NN, 1)) 
    X_train = X_train/float(len(chars))
    y_train = np_utils.to_categorical(test_df.result_int)
    print(" i am in preparing end")

    #####################################################
    # Model
    print("i am in model start")
    model = Sequential()
    model.add(LSTM(256, input_shape = (X_train.shape[1], X_train.shape[2]))) # 256 hidden nodes
    model.add(Dropout(0.2))#This technique is applied in the training phase to reduce overfitting effects.# https://www.python-course.eu/neural_networks_with_dropout.php
    model.add(Dense(y_train.shape[1], activation = 'softmax'))
    # model.summary()

    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train, y_train, epochs = 50, batch_size = 2000)
    print("i am in model end")
    ######################################################
    # Saving the model 
    print("i am in saving model start")
    pickle_op_dict = dict()
    pickle_op_dict['char_to_integer']= char_to_integer
    pickle_op_dict['integer_to_char']= integer_to_char
    pickle_op_dict['chars']          = chars
    pickle_op_dict['len_input_NN']   = len_input_NN
    pickle_op_dict['integer_to_cat'] = integer_to_cat
    pickle_op_dict['cat_to_integer'] = cat_to_integer
    pickle_dict = 'Models/ML_LCT_'+type_+'_dict.pkl'
    pickle_out = open(pickle_dict,"wb")
    pickle.dump(pickle_op_dict, pickle_out)
    pickle_out.close()
    print("saved pickle")
    model_file= 'Models/ML_LCT_'+type_+'_Model.h5'
    model.save(model_file)
    print('i am in saving model end')
    return (test_df,X_train,model,integer_to_cat)

#################################################
# Model Accuracy 
def pred_(Input,model):
    X_input = np.reshape(Input, (1, len_input_NN, 1))
    y_pred = model.predict(X_input, verbose=0)
    return np.argmax(y_pred)
def model_accuracy(test_df,X_train,model):
    df_ = test_df.copy()
    df_['pred']=''
    # test1.head()

    for index,value in enumerate(X_train):
        df_['pred'][index] = pred_(value,model)
    return df_

def generate_report(type_):
    data_ = train_model(type_)
    test_df = data_[0]
    X_train = data_[1]
    model = data_[2]
    integer_to_cat = data_[3]
    df_ = model_accuracy(test_df,X_train,model)
    target_names =[]
    for key_,value_ in integer_to_cat.items():
        target_names.append(value_)

    report = classification_report(list(df_['result_int']), list(df_['pred']),target_names=target_names, output_dict=True)
    return report

 i am in library start


Using TensorFlow backend.


 i am in library end


# Flask Application

In [ ]:
from flask import Flask, request, render_template, session, redirect
### working code -- which displays the DataFrame

app = Flask(__name__)

@app.route('/')
def my_form():
    return render_template('Train_ML_Report_index.html')

@app.route('/', methods=("POST", "GET"))
def html_table():
    K.clear_session()

#     type_ = 'location'
    type_ = request.form['schemas_']
    data_frame = pd.DataFrame(generate_report(type_)).transpose()

    return render_template('Train_ML_Report.html',  tables=[data_frame.to_html(classes='data')])

if __name__ == '__main__':
    app.run(host = '127.0.0.2', port='5002',threaded=False) # host = '127.0.0.2', port='5002'

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.2:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jul/2020 17:23:34] "GET / HTTP/1.1" 200 -


 i am in connection database start
 i am in connection database end
 i am in char_num building start
 i am in char_num building end
 i am in assigning start
 i am in assigning end
 i am in category start
 i am in category end
 i am in preparing start
 i am in preparing end
i am in model start
Epoch 1/50
6897/6897 [==============================] - 2s 327us/step - loss: 0.2447 - accuracy: 0.9333
Epoch 2/50
6897/6897 [==============================] - 1s 201us/step - loss: 0.2433 - accuracy: 0.9333
Epoch 3/50
6897/6897 [==============================] - 1s 201us/step - loss: 0.2375 - accuracy: 0.9336
Epoch 4/50
6897/6897 [==============================] - 1s 201us/step - loss: 0.2299 - accuracy: 0.9334
Epoch 5/50
6897/6897 [==============================] - 1s 202us/step - loss: 0.2180 - accuracy: 0.9331
Epoch 6/50
6897/6897 [==============================] - 1s 202us/step - loss: 0.2126 - accuracy: 0.9328
Epoch 7/50
6897/6897 [==============================] - 1s 202us/step - loss: 0.20

127.0.0.1 - - [13/Jul/2020 17:28:01] "POST / HTTP/1.1" 200 -


# Testing

In [11]:
# conn = psycopg2.connect("dbname=postgres user=postgres password=Msjobs@123")
# type_='location'
# def select_data(type_):
#     cur = conn.cursor()
#     sql_command = "select value_,key_ from public.lct_ml_key_value where type_ ='" +type_+"';"
#     sql_command = "select value_,key_ from public.lct_ml_key_value where type_ = 'location';"

#     data = pd.read_sql(sql_command, conn)
#     conn.commit()
#     cur.close()
#     return data  

In [3]:
# #Renaming the columns
# # type_ = 'location'
# test_df = select_data(type_).rename(columns={"value_": "data","key_": "result"})
# print(" i am in connection database end")
# #####################################################
# # Handling varity of characters in the upcoming inputs
# print(" i am in char_num building start")
# listchars_ = ' "klmnopqrstvuwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890-=+_`~[]{}\|:;,.<>?/!@#$%^&*()ïáúãüóíéñÁäöôàÉ'
# chars = sorted(list(set(listchars_)))


In [6]:
# char_to_integer = []
# for integer, char in enumerate(chars,start=1):
#     char_to_integer.append((char, integer))
# char_to_integer = dict(char_to_integer)

# integer_to_char = []
# for integer, char in enumerate(chars,start=1):
#     integer_to_char.append((integer,char))
# integer_to_char = dict(integer_to_char)
# print(" i am in char_num building end")
# #####################################################
# # Assigning Numeric to all the values
# print(" i am in assigning start")
# len_input_NN = 30  #this determined the number of Input to the nueral network model
# test_df["data_int"]= ""

# test_df

In [ ]:
# for i in range(0, (len(test_df.data))):
#     data_chr_int =[0]*len_input_NN
#     data_ = str(test_df.data[i])
#     i_ = 0 
#     for char in data_:
#         data_chr_int[i_] = char_to_integer[char]
#         if i_ >= len_input_NN-1:
#             break
#         i_+=1
#     test_df["data_int"][i] = data_chr_int
# print(" i am in assigning end")
# test_df

In [4]:
# data_chr_int =[0]*30
# i_=0
# for i in "dinesh":
#     try:
#         data_chr_int[i_] = char_to_integer[i]
#     except :
#         data_chr_int[i_] = 0
#     print(i)
#     if i_ >= 30-1:
#             break
#     i_+=1
# data_chr_int

In [1]:
# def generate_input_sequence(data_):
#     data_chr_int =[0]*len_input_NN
#     i_ = 0 
#     for char in data_:
#         data_chr_int[i_] = char_to_integer[char]
#         if i_ >= len_input_NN-1:
#             break
#         i_+=1
#     data_chr_int = np.reshape(data_chr_int, (1, len_input_NN, 1)) 
#     data_chr_int = data_chr_int/float(len(chars))
#     return data_chr_int
# # generate_input_sequence("dinesh")

# def generate_output_sequence(input_sequence):
#     Test_char = []
#     for i in input_sequence:
#         if i == 0:
#             break
#         Test_char.append(integer_to_char[round(list(i*len(chars))[0],0)])
#     Test_char = ''.join(Test_char)
#     return Test_char   
# # generate_output_sequence(input_sequence)

# def generate_random_sequence():
#     random_number = np.random.randint(0, len(X_train)-1)
#     input_sequence = X_train[random_number]
#     return input_sequence
# #     print(input_sequence)

In [6]:
# random_number = np.random.randint(0, len(X_train)-1)
# input_sequence = X_train[random_number]
# print(input_sequence)

In [2]:
# random_sequence = generate_input_sequence("23")[0]
# input_sequence

In [21]:
# generate_output_sequence(input_sequence)

In [1]:
# Input for this file is Type so that it will generate the model which is required accordingly can be used for reinforcement of the ML model in the architecture
# type_ = 'location'
# %%time
# #####################################################
# #Library

# import warnings
# warnings.filterwarnings("ignore")

# import pandas as pd
# import numpy as np
# import psycopg2,os,csv

# from IPython.display import display
# pd.options.display.max_columns = None

# from keras import backend as K
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout,Embedding
# from keras.utils import np_utils
# from keras.utils import plot_model

# from sklearn.metrics import classification_report

# from pickle import dump
# import pickle
# #######################################################
# #Connecting to Database
# conn = psycopg2.connect("dbname=postgres user=postgres password=Msjobs@123")

# def select_data(type_):
#     cur = conn.cursor()
#     sql_command = "select value_,key_ from public.lct_ml_key_value where type_ ='" +type_+"';"
# #     sql_command = "select value_,key_ from public.lct_ml_key_value where type_ = 'location';"
#     data = pd.read_sql(sql_command, conn)
#     conn.commit()
#     cur.close()
#     return data  

# #Renaming the columns
# # type_ = 'location'
# test_df = select_data(type_).rename(columns={"value_": "data","key_": "result"})

# #####################################################
# # Handling varity of characters in the upcoming inputs
# listchars_ = ' "abcdefghijklmnopqrstvuwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890-=+_`~[]{}\|:;,.<>?/!@#$%^&*()áúãüóíéñÁäöôàÉ'
# chars = sorted(list(set(listchars_)))

# char_to_integer = []
# for integer, char in enumerate(chars,start=1):
#     char_to_integer.append((char, integer))
# char_to_integer = dict(char_to_integer)

# integer_to_char = []
# for integer, char in enumerate(chars,start=1):
#     integer_to_char.append((integer,char))
# integer_to_char = dict(integer_to_char)

# #####################################################
# # Assigning Numeric to all the values
# len_input_NN = 30  #this determined the number of Input to the nueral network model
# test_df["data_int"]= ""

# for i in range(0, (len(test_df.data))):
#     data_chr_int =[0]*len_input_NN
#     data_ = str(test_df.data[i])
#     i_ = 0 
#     for char in data_:
#         data_chr_int[i_] = char_to_integer[char]
#         if i_ >= len_input_NN-1:
#             break
#         i_+=1
#     test_df["data_int"][i] = data_chr_int
    
    
# #####################################################
# # Assigning the category to index and index to category for predicting the category in future using index
# result_char = (list(set(test_df.result)))
# cat_to_integer = []
# integer_to_cat = []

# for integer, char in enumerate(result_char,start=0):
#     cat_to_integer.append((char, integer))
# cat_to_integer = dict(cat_to_integer)

# for integer, char in enumerate(result_char,start=0):
#     integer_to_cat.append((integer, char))
# integer_to_cat = dict(integer_to_cat)

# test_df["result_int"] = ""
# for i in range(0, (len(test_df.result))):
#     test_df["result_int"][i] = cat_to_integer[test_df['result'][i]]
     
    
# #####################################################
# # Preparing the data for the Nueral Network model 
# X_train =[]

# for i in test_df.data_int:
#     X_train.append(i)
    
# # Reshape X_train and normalize
# X_train = np.reshape(X_train, (len(X_train), len_input_NN, 1)) 
# X_train = X_train/float(len(chars))
# y_train = np_utils.to_categorical(test_df.result_int)


# #####################################################
# # Model
# model = Sequential()
# model.add(LSTM(256, input_shape = (X_train.shape[1], X_train.shape[2]))) # 256 hidden nodes
# model.add(Dropout(0.2))#This technique is applied in the training phase to reduce overfitting effects.# https://www.python-course.eu/neural_networks_with_dropout.php
# model.add(Dense(y_train.shape[1], activation = 'softmax'))
# # model.summary()

# model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# model.fit(X_train, y_train, epochs = 100, batch_size = 100)

# ######################################################
# # Saving the model 
# pickle_op_dict = dict()
# pickle_op_dict['char_to_integer']= char_to_integer
# pickle_op_dict['integer_to_char']= integer_to_char
# pickle_op_dict['chars']          = chars
# pickle_op_dict['len_input_NN']   = len_input_NN
# pickle_op_dict['integer_to_cat'] = integer_to_cat
# pickle_op_dict['cat_to_integer'] = cat_to_integer

# pickle_dict = 'ML_LCT_'+type_+'_dict.pkl'
# pickle_out = open(pickle_dict,"wb")
# pickle.dump(pickle_op_dict, pickle_out)
# pickle_out.close()

# model_file= 'ML_LCT_'+type_+'_Model.h5'
# model.save(model_file)

In [3]:
# random_sequence = generate_random_sequence()
# # print(random_sequence)
# generate_output_sequence(random_sequence)

In [4]:
# %%time
# X_input = np.reshape(random_sequence, (1, len(input_sequence), 1))
# y_pred = model.predict(X_input, verbose=0)
# # y_pred

# index = np.argmax(y_pred)
# print(index,integer_to_cat[index])
# # integer_to_cat[index],integer_to_cat[index],integer_to_cat[index+1],index

In [5]:
# # dataframe_ = []
# data = [['test1','test1','test1']]
# dataframe_ = pd.DataFrame([],columns=['index','value','column'])
# dataframe_
# for index,i in enumerate(y_pred[0],start=0):
# #     if index == 54:
# #         break
#     data = {'index':index,'value':round(i*100,2),'column':integer_to_cat[index]}
#     dataframe_ = dataframe_.append(data,ignore_index=True)    

# dataframe_.sort_values(by=['value'],ascending=False)

In [4]:
# X_train

In [5]:
# print(" i am in connection database start")
# import psycopg2
# conn = psycopg2.connect("dbname=postgres user=postgres password=Msjobs@123")

# def select_data(type_):
#     cur = conn.cursor()
#     sql_command = "select value_,key_ from public.lct_ml_key_value where type_ ='" +type_+"';"
# #     sql_command = "select value_,key_ from public.lct_ml_key_value where type_ = 'location';"
#     data = pd.read_sql(sql_command, conn)
#     conn.commit()
#     cur.close()
#     return data  